## 전체 코드

In [ ]:
import torch
import numpy
import re
import pickle
import random
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# transformers, genai 임포트
from transformers import BertForSequenceClassification, AutoTokenizer, AutoModel
import genai
#from genai.schemas import GenerateParams
#from genai.models import GenerativeModel
import sys
import os
sys.path.append(os.path.abspath(".."))  # 현재 디렉터리의 상위 경로 추가

from config.api_keys import gemini_key
import google.generativeai as genai


# -------------------------------
#  첫 번째 감정 분류 (KoBERT)
# -------------------------------
class EmotionClassifier:
    def __init__(self, model_path="monologg/kobert", num_labels=7, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model.to(self.device)

        self.label_to_emotion = {
            0: "중립",
            1: "놀람",
            2: "분노",
            3: "슬픔",
            4: "행복",
            5: "혐오",
            6: "공포"
        }

    def load_model(self, model_file):
        """학습된 .pth 파라미터를 로드"""
        self.model.load_state_dict(torch.load(model_file, map_location=self.device))

    def preprocess_text(self, text):
        """간단한 전처리: 특수문자 제거"""
        return re.sub("[^0-9a-zA-Z가-힣\s+]", "", text)

    def predict_emotion(self, text):
        """문장 -> 감정 레이블"""
        cleaned_text = self.preprocess_text(text)
        encoded_input = self.tokenizer(
            cleaned_text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=128
        )
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}

        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**encoded_input)
            predicted_label = outputs.logits.argmax(dim=1).item()

        return self.label_to_emotion[predicted_label]

# -----------------------------
#  E5 임베딩 클래스
# -----------------------------
class E5Embedder:
    def __init__(self):
        # E5 모델 (intfloat/e5-large) 불러오기
        self.tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-large")
        self.model = AutoModel.from_pretrained("intfloat/e5-large")

    def get_embedding(self, text):
        """텍스트를 E5 임베딩 벡터로 변환"""
        # 원하는 전처리 규칙(예: 'query: ', 'passage: ') 등을 적용할 수도 있음
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        embedding = outputs.last_hidden_state.mean(dim=1).detach()
        return embedding  # shape: [1, hidden_dim]

# -----------------------------
#  노래 추천(코사인 유사도)
# -----------------------------
class SongRecommender:
    """
    df: 노래 정보 DataFrame (['title','artist','cleaned_lyrics','emotion','embedding'])
        - embedding: E5Embedder로 미리 구한 텐서
    """
    def __init__(self, df):
        self.df = df

    def recommend_song(self, diary_embedding, emotion):
        """
        일기 임베딩(diary_embedding), 감정(emotion)을 받아
        동일 감정의 곡 중 코사인 유사도가 가장 높은 노래 추천
        """
        # 감정이 동일한 노래만 필터링
        filtered_df = self.df[self.df['emotion'] == emotion]

        similarities = []
        # diary_embedding이 torch.Tensor라면 numpy 변환
        if isinstance(diary_embedding, torch.Tensor):
            diary_emb_np = diary_embedding.numpy()
        else:
            diary_emb_np = diary_embedding

        for _, row in filtered_df.iterrows():
            # 곡 임베딩
            song_emb = row['embedding']
            # 만약 song_emb도 torch.Tensor라면 numpy 변환
            if isinstance(song_emb, torch.Tensor):
                song_emb = song_emb.numpy()

            sim = cosine_similarity(diary_emb_np, song_emb)
            # sim.shape: (1, 1)
            similarities.append((
                row['title'],
                row['artist'],
                row['cleaned_lyrics'],
                sim[0][0]  # 유사도 값
            ))

        if similarities:
            # 유사도 내림차순 정렬
            best_match = sorted(similarities, key=lambda x: x[3], reverse=True)[0]
            return best_match  # (title, artist, lyrics, similarity)
        else:
            return None

# -----------------------------
#  Gemini 설정 & 일기 초안 
# -----------------------------

# Gemini API 설정
genai.configure(api_key=gemini_key)

# Gemini 모델 선택
gemini_model = genai.GenerativeModel('gemini-2.0-flash')

chat_history = []

def ask_gemini(user_response, emotion, max_history_length=5):
    """
    사용자 입력 + 감정 → 프롬프트 구성 → Gemini 대화
    """
    global chat_history

    conversation_history = "\n".join(chat_history)
    
    full_prompt = (
        f"{conversation_history}"
        f"\nUser (감정: {emotion}): {user_response}\n\n"
        "사용자는 일기를 꾸준히 쓰고 싶어하는 사람입니다. "
        "한 번 써보고 끝이 아니라, 매일 재미를 느끼며 계속 작성할 수 있도록 동기를 부여해주세요. "
        "친근하고 공감하는 어조로, 사용자의 감정을 이해하고, 감정을 조금 더 탐색할 수 있는 질문을 한 개만 만들어주세요. "
        "답변은 2~3문장으로 간결하게 유지하고, 예시는 1개 정도만 들어주세요. "
        "이전 대화 내용도 반영해주세요."
    )

    response = gemini_model.generate_content(full_prompt)

    # 대화 기록 저장
    chat_history.append(f"User (감정: {emotion}): {user_response}")
    chat_history.append(f"Gemini: {response.text}")

    # 너무 길어지면 초기 기록 삭제
    if len(chat_history) > max_history_length * 2:
        chat_history = chat_history[-(max_history_length * 2):]

    return response.text

def generate_diary_draft(chat_history):
    """
    전체 대화 내용을 요약하여 '일기 초안' 작성
    """
    full_conversation = "\n".join(chat_history)
    
    prompt = f"""
    아래 대화 내용을 바탕으로, 사용자의 감정과 상황이 잘 드러나는 일기 초안을 작성해 주세요.
    문맥이 자연스럽고 핵심 내용이 잘 담기도록 정리하되, 
    '앞으로도 매일 일기를 쓰고 싶어지는' 동기가 될 만한 따뜻하고 희망적인 문장들을 포함해주세요.
    간결하면서도, 사용자가 자신을 돌아볼 수 있는 한두 문장과
    내일 혹은 다음 일기를 위한 작은 다짐이나 기대감이 느껴지도록 작성해 주세요.

    대화 내용:
    "{full_conversation}"
    """

    response = gemini_model.generate_content(prompt)
    return response.text

# 수정 반영해서 최종 일기 만들기

def incorporate_user_changes(original_draft, user_changes):
    """
    - original_draft: Gemini가 생성한 '초기 일기 초안'
    - user_changes: 사용자가 수정하고 싶은 내용 (여러 줄)
    이 둘을 합쳐, 최종 버전을 다시 Gemini에게 요청하여 생성.
    """
    prompt = f"""
    아래는 일기 초안입니다:
    ===
    {original_draft}
    ===

    사용자가 다음과 같은 수정 사항을 제시했습니다:
    ===
    {user_changes}
    ===

    위 수정 사항을 충실히 반영하면서도, 전체 글이 자연스럽게 이어지도록
    최종 일기를 만들어 주세요. 초안의 분위기와 톤을 유지하되,
    사용자의 수정이 우선 적용되어야 합니다.
    초안 처럼 '앞으로도 매일 일기를 쓰고 싶어지는' 동기가 될 만한 따뜻하고 희망적인 문장들을 포함해주세요.
    간결하면서도, 사용자가 자신을 돌아볼 수 있는 한두 문장과
    내일 혹은 다음 일기를 위한 작은 다짐이나 기대감이 느껴지도록 작성해 주세요.
    
    """
    response = gemini_model.generate_content(prompt)
    return response.text

# -----------------------------
#  기분 전환 활동 & 상품 추천
# -----------------------------
def suggest_activities_and_products(diary_text, emotion):
    """
    일기 초안을 바탕으로,
    - 기분전환 활동 (2~3개)
    - 상품(책, 소품, 음식 등) 추천
    을 Gemini에게 요청.
    """
    prompt = f"""
    당신은 일기를 작성하는 사람을 돕는 조력자입니다.
    아래는 사용자의 일기 초안과 감정 상태입니다.
    
    일기: "{diary_text}"
    감정: {emotion}
    
    이 사용자에게 적합한 기분전환 활동을 두세 가지 정도 제안하고,
    도움이 될 만한 상품이나 서비스를 하나 이상 추천해주세요.
    답변은 간결하게, 핵심만 정리해 주세요.사용자 입장에서 어렵지 않게 시도해볼 수 있는 활동을 알려주세요.
    """
    response = gemini_model.generate_content(prompt)
    return response.text

# -----------------------------
#  메인 실행부
# -----------------------------
if __name__ == "__main__":
    #  KoBERT 감정 분류 모델
    emotion_classifier = EmotionClassifier(model_path="monologg/kobert", num_labels=7)
    emotion_classifier.load_model("./new_data_test.pth")  # 실제 .pth 경로

    # E5 임베더(일기 초안 등 임베딩용)
    e5_embedder = E5Embedder()

    # 노래 데이터 로드 (E5 임베딩이 저장된 DataFrame)
    df = pd.read_pickle("./trot_embeddings_emotion.pkl")
    # df 컬럼: ['title','artist','cleaned_lyrics','emotion','embedding']
    recommender = SongRecommender(df)

    print("대화를 시작합니다. 종료하려면 'exit'를 입력하세요.")

    # 대화 루프
    while True:
        user_response = input("User: ")
        if user_response.lower() == "exit":
            print("대화를 종료합니다.")
            break
        
        #  사용자 입력 감정 분석
        emotion = emotion_classifier.predict_emotion(user_response)

        #  Gemini 질의
        follow_up_question = ask_gemini(user_response, emotion)
        print("Gemini:", follow_up_question)
        
        # (옵션) 대화 기록 모니터링
        print("\n[대화 기록 확인]")
        for chat in chat_history:
            print(chat)
        print("\n----------------------")

 # 일기 초안 생성
    # ---------------------------
    print("\n==== 전체 대화 요약 -> 일기 초안 생성 ====")
    diary_draft = generate_diary_draft(chat_history)
    print("초기 일기 초안:\n", diary_draft)

    # ---------------------------
    #  사용자 일기 초안 보고 수정 (멀티라인)
    # ---------------------------
    print("\n일기 초안을 수정하시겠어요?")
    print("여러 줄 입력 가능, 빈 줄(엔터) 입력 시 종료.")
    print("수정 없이 넘어가려면 'SKIP'이라고 입력.")
    
    lines = []
    while True:
        line = input()
        if line.strip().lower() == "skip":
            # 수정사항 없이 그대로
            user_changes = ""
            break
        elif line.strip() == "":
            # 빈 줄이면 수정 입력 끝
            user_changes = "\n".join(lines)
            break
        else:
            lines.append(line)
    
    if user_changes:
        #  수정내용이 있다면, Gemini에 "초안+수정"으로 최종 일기 재생성
        final_diary = incorporate_user_changes(diary_draft, user_changes)
    else:
        #  수정 없으면 초안 그대로
        final_diary = diary_draft

    # ---------------------------
    # 최종 일기 감정 & 노래추천
    # ---------------------------
    print("\n[최종 일기 내용]:")
    print(final_diary)
    
    final_emotion = emotion_classifier.predict_emotion(final_diary)
    print(f"\n최종 일기 감정: {final_emotion}")

    diary_emb = e5_embedder.get_embedding(final_diary)
    recommended_song = recommender.recommend_song(diary_emb, final_emotion)

    print("\n==== 노래 추천 ====")
    if recommended_song is not None:
        title, artist, lyrics, similarity = recommended_song
        print(f"🎵 제목: {title}")
        print(f"👤 가수: {artist}")
        print(f"📜 가사:\n{lyrics}")
        print(f"🔗 유사도 점수: {similarity:.4f}")
    else:
        print("추천할 노래가 없습니다.")

    # ---------------------------
    # 5) 기분전환 활동 & 상품 추천
    # ---------------------------
    print("\n==== 기분전환 활동 & 상품 추천 ====")
    suggestions = suggest_activities_and_products(final_diary, final_emotion)
    print(suggestions)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\user\AppData\Local\Temp\ipykernel_16708\2107337977.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializat

대화를 시작합니다. 종료하려면 'exit'를 입력하세요.
Gemini: 와, 벌써 4번째 일기라니! 꾸준히 쓰는 모습이 정말 대단해요. 매일 일기를 쓰는 게 쉽지 않은데, 놀라운 끈기를 보여주고 있네요. 혹시 4번째 일기를 쓰려고 하니, 처음 일기를 쓰기로 결심했을 때와 비교해서 어떤 점이 달라졌나요? 예를 들어, 글쓰기가 더 편안해졌을 수도 있고, 새로운 관점을 발견했을 수도 있겠네요.


[대화 기록 확인]
User (감정: 놀람): 4번째 일기를 쓰려고 해
Gemini: 와, 벌써 4번째 일기라니! 꾸준히 쓰는 모습이 정말 대단해요. 매일 일기를 쓰는 게 쉽지 않은데, 놀라운 끈기를 보여주고 있네요. 혹시 4번째 일기를 쓰려고 하니, 처음 일기를 쓰기로 결심했을 때와 비교해서 어떤 점이 달라졌나요? 예를 들어, 글쓰기가 더 편안해졌을 수도 있고, 새로운 관점을 발견했을 수도 있겠네요.


----------------------
Gemini: 와, 점심을 정말 맛있게 드셨다니 저도 기분이 좋아지네요! 맛있는 음식을 먹는 건 정말 큰 행복이죠. 4번째 일기도 쓰실 만큼 꾸준하시니, 혹시 오늘 점심 메뉴에 대한 행복한 기억을 일기에 담아보는 건 어떠세요? 예를 들어, "오늘따라 유난히 맛있었던 [메뉴 이름] 덕분에 오후 내내 기분 좋게 일기를 쓸 수 있었다." 같이요!


[대화 기록 확인]
User (감정: 놀람): 4번째 일기를 쓰려고 해
Gemini: 와, 벌써 4번째 일기라니! 꾸준히 쓰는 모습이 정말 대단해요. 매일 일기를 쓰는 게 쉽지 않은데, 놀라운 끈기를 보여주고 있네요. 혹시 4번째 일기를 쓰려고 하니, 처음 일기를 쓰기로 결심했을 때와 비교해서 어떤 점이 달라졌나요? 예를 들어, 글쓰기가 더 편안해졌을 수도 있고, 새로운 관점을 발견했을 수도 있겠네요.

User (감정: 행복): 점심을 너무 맛있게 먹었어 
Gemini: 와, 점심을 정말 맛있게 드셨다니 저도 기분이 좋아지네요! 맛있는 음식을 먹는 건 정말 큰 행복이죠. 4번째 일기도 쓰